# Day 9
## Part 1


In [5]:
from advent import read_input, Point
import itertools

def parse_data(s):
    return [
        Point(*map(int, line.split(","))) 
        for line in s.strip().splitlines()
    ]

test_data = parse_data("""7,1
11,1
11,7
9,7
9,5
2,5
2,3
7,3
""")

test_data

[Point(x=7, y=1),
 Point(x=11, y=1),
 Point(x=11, y=7),
 Point(x=9, y=7),
 Point(x=9, y=5),
 Point(x=2, y=5),
 Point(x=2, y=3),
 Point(x=7, y=3)]

In [6]:
def area(p1, p2):
    return abs(p1.x - p2.x + 1) * abs(p1.y - p2.y + 1)

def part_1(data):
    return max(
        area(p1, p2) 
        for p1, p2 in itertools.combinations(data, 2)
    )

assert part_1(test_data) == 50

In [7]:
data = parse_data(read_input())

part_1(data)

4777824480

## Part 2

I'm sure we've had something like this before.

In [17]:
def boundaries(data):
    return[
        (p1, p2) 
        for p1, p2 in itertools.combinations(data, 2)
        if p1.x == p2.x or p1.y == p2.y
    ]

len(boundaries(data))

496

I don't know how to do this, I'll have a think.